In [1]:
import pandas as pd
import numpy as np

# from api_data import ApiData
from api_executer import ApiExecuter
from api_requester import ApiRequester
from api_parser import ApiParser
from FUNCTIONS import get_current_location

In [ ]:
location = get_current_location()
if location:
    latitude, longitude = location
    print(f'Current Latitude: {latitude}, Longitude: {longitude}')
else:
    print('Failed to determine current location.')

In [2]:
from api_route import ApiRoute
current_position = {'longitude': 130.844161,
                    'latitude': -12.456694}
final_position = {'longitude': 130.890346,
                  'latitude': -12.432393}
delta_spacing = 2000.0 # in meters
number_sites = 2
route = ApiRoute()
# dataframe = route.get_route_data(current_position)
dataframe = route.get_route_data(current_position, number_sites=number_sites, delta_spacing=delta_spacing, print_is_requested=True)
# dataframe = route.get_route_data(current_position=current_position, final_position=final_position, number_sites=number_sites)

Nearest point index: 8
    longitude   latitude  altitudeSmoothed  inclinationSmoothed  cumDistance  \
0  130.844161 -12.456694         16.350559            -0.024998   407.095561   
1  130.849361 -12.440116          8.870473             0.009980  2407.095561   

   maxSpeed      theta  cumDistanceCut  
0        50  44.136411             0.0  
1        80  17.935500          2000.0  


In [3]:
api_parser = ApiParser()
api_requester = ApiRequester(parser=api_parser)
api_executer = ApiExecuter(requester=api_requester)

Response status from siteinfo: OK.
Current sites' info has been retrieved. 
         name  longitude  latitude  altitude
site_id                                    
585395    aa        134       -13        32
585396    df        134       -13        16


In [4]:
api_requester.get_site_info(print_is_requested=True)

Response status from siteinfo: OK.
Site information have been retrieved: 
 {
  "status": "OK",
  "statuscode": 200,
  "payload": {
    "solarforecast": {
      "sites": {
        "585395": {
          "altitude": 32,
          "azimuth": 0,
          "cddctout": null,
          "hddctin": null,
          "hddctout": null,
          "horizon": [
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0
          ],
          "id": 585395,
          "inclination": 0,
          "last_update": "2023-09-03 16:51:04",
          "latitude": -13,
          "longitude": 134,
          "name": "aa",
          "registered": "2023-09-03 16:49:37",
          "timezone": "Australia/Darwin",
          "utc_offset": "UTC+9.5"
        },
        "585396": {
          "altitude": 16,
          "azimuth": 0,
          "cddctout": null,
          "hddctin": null,
          "hddctout": n

,name,longitude,latitude,altitude
site_id,,,,
585395,aa,134,-13,32
585396,df,134,-13,16


In [ ]:
api_executer.add_sites(dataframe, print_is_requested=True)

In [ ]:
# api_requester.get_site_delete(585394, print_is_requested=True)
api_executer.delete_all_sites()

In [6]:
position = {
    'latitude': -13.0,
    'longitude': 134.0
}
api_requester.get_site_edit(585396, True, name="aa")
api_requester.get_site_edit(585395, True, position=position)
api_requester.get_site_edit(585396, True, name="df", position=position)

Site with id 585396 has been edited: New name: aa.  
         name  longitude  latitude  altitude
site_id                                    
585395    aa        134       -13        32
585396    aa        134       -13        16
Site with id 585395 has been edited: New position: Longitude: 134.0, latitude: -13.0.  
         name  longitude  latitude  altitude
site_id                                    
585395    aa        134       -13        32
585396    aa        134       -13        16
Site with id 585396 has been edited: New name: df. New position: Longitude: 134.0, latitude: -13.0.  
         name  longitude  latitude  altitude
site_id                                    
585395    aa        134       -13        32
585396    df        134       -13        16


In [7]:
# forecasts = api_requester.get_solar_forecast()
forecasts = api_requester.get_solar_forecast_cloudmove()
print(forecasts)

Response status from getforecast_cloudmove: OK.
Solar forecast CloudMove have been retrieved.
                             gk      tt  gh  dh  bh  dni  e
site_id time                                               
585395  2023-09-03 17:30:00   0  16.750   0   0   0    0  0
        2023-09-03 17:45:00   0  16.675   0   0   0    0  0
        2023-09-03 18:00:00   0  16.600   0   0   0    0  0
        2023-09-03 18:15:00   0  16.425   0   0   0    0  0
        2023-09-03 18:30:00   0  16.250   0   0   0    0  0
        2023-09-03 18:45:00   0  16.075   0   0   0    0  0
        2023-09-03 19:00:00   0  15.900   0   0   0    0  0
        2023-09-03 19:15:00   0  15.750   0   0   0    0  0
        2023-09-03 19:30:00   0  15.600   0   0   0    0  0
        2023-09-03 19:45:00   0  15.450   0   0   0    0  0
        2023-09-03 20:00:00   0  15.300   0   0   0    0  0
        2023-09-03 20:15:00   0  15.325   0   0   0    0  0
        2023-09-03 20:30:00   0  15.350   0   0   0    0  0
      

In [ ]:
# Extract all 'rr' values for all site_id and time combinations
# rr_values = forecasts.loc[:, ('rr', slice(None), slice(None))].droplevel('variable')
rr_values = forecasts.loc[:, ('rh')]
rr_values = rr_values.to_frame()
print(rr_values)
print(rr_values.max(), rr_values.min())
# # Assuming your MultiIndex DataFrame is named "forecast_data"
# pivot_table = rr_values.pivot(index='time', columns='site_id', values='tt')
# print(pivot_table)
rr_values.to_csv('rh_table.csv')

# print(rr_values)